In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/pointwiseclassification/classification_dataset.tsv
/kaggle/input/pointwiseclassificationcheckpoints/checkpoint-78174/config.json
/kaggle/input/pointwiseclassificationcheckpoints/checkpoint-78174/trainer_state.json
/kaggle/input/pointwiseclassificationcheckpoints/checkpoint-78174/training_args.bin
/kaggle/input/pointwiseclassificationcheckpoints/checkpoint-78174/tokenizer.json
/kaggle/input/pointwiseclassificationcheckpoints/checkpoint-78174/tokenizer_config.json
/kaggle/input/pointwiseclassificationcheckpoints/checkpoint-78174/pytorch_model.bin
/kaggle/input/pointwiseclassificationcheckpoints/checkpoint-78174/scheduler.pt
/kaggle/input/pointwiseclassificationcheckpoints/checkpoint-78174/special_tokens_map.json
/kaggle/input/pointwiseclassificationcheckpoints/checkpoint-78174/optimizer.pt
/kaggle/input/pointwiseclassificationcheckpoints/checkpoint-78174/vocab.txt
/kaggle/input/pointwiseclassificationcheckpoints/checkpoint-78174/rng_state.pth


In [2]:
checkpoint = 'google/electra-small-discriminator'

In [3]:
!pip install datasets

In [4]:
classify_df = pd.read_csv('/kaggle/input/pointwiseclassification/classification_dataset.tsv', sep='\t')
classify_df.rename(columns={'esci_label': 'label'}, inplace=True)

In [5]:
from sklearn.model_selection import train_test_split
train = []
test = []
for label, _df in classify_df.groupby(['label']):
    train_df, test_df = train_test_split(_df,test_size=0.2, random_state=5)
    train.append(train_df)
    test.append(test_df)

In [6]:
train_df = pd.concat(train, ignore_index=True)
test_df = pd.concat(test, ignore_index=True)

In [7]:
upsampled_train = []
sample_frac = {'exact': 0.8, 'substitute': 1.0, 'irrelevant': 2.0, 'complement': 7.8}
for label, _df in train_df.groupby(['label']):
    upsampled_train.append(_df.sample(frac=sample_frac[label], replace=True, random_state=5))

In [8]:
upsample_train_df = pd.concat(upsampled_train, ignore_index=True)

In [9]:
upsample_train_df.label.value_counts()

complement    253032
exact         218347
substitute    214369
irrelevant    211290
Name: label, dtype: int64

In [10]:
pd.set_option('display.max_colwidth', None)

In [11]:
train_df[train_df['label'] == 'substitute'].sample(n=5)

,input,label
568055,"sew patterns for beginners us [SEP] Wrights Simplicity Learn to Sew-18, 18"" Doll Clothes us Wright's Create accessories for you and your doll! Choose your fabrics, trace pattern template, cut fabric and sew! This 6x8. 5 inch package contains five exclusive pattern templates and three bonus projects for 18 inch dolls\nRecommended for ages 8 and up\nPackage Dimensions: 0.254 L x 15.493 H x 11.937 W (centimeters)\nPackage Weight: 0.018 kilograms 18"" Doll Clothes",substitute
568880,"primal vanilla powder us [SEP] Garden of Life Raw Organic Protein Vanilla Powder, 20 Servings: Certified Vegan, Gluten Free, Organic, Non-GMO, Plant Based Sugar Free Protein Shake with Probiotics & Enzymes, 4g BCAAs, 22g Protein us Garden of Life SMOOTH, CREAMY TEXTURE: Get 22g of muscle-building Certified Organic and Non-GMO protein made from peas and raw sprouted seeds, grains and legumes—plus 3 Billion CFU of live probiotics & 13 digestive enzymes—for a delicious blender free protein smoothie\nCOMPLETE PLANT BASED PROTEIN: Protein and BCAAs after a workout can help your body rebuild broken down muscle and reduce recovery time; Raw Organic Protein is a complete plant protein with all essential amino acids and 4g of naturally occurring BCAA\nFOUNDATIONAL, ORGANIC FOODS: We combine the cleanest vegan proteins, including organic peas and 13 organic sprouted grains, seeds and legumes, with fat-soluble vitamins A, D, E and K, probiotics and enzymes, to help promote digestive and immune health\nEASY TO DIGEST: Heat can denature proteins, reducing their availability to your body; Raw Organic Protein is made at low temperatures, preserving its complete amino acid integrity—with live probiotics and protein-digesting enzymes for proper absorption\nEMPOWERING EXTRAORDINARY HEALTH - We start with what goes IN our products; whole food ingredients and rigorous standards because we believe in truly clean products you can trust. We do it because it’s the only way we know how to deliver on our promise Vanilla",substitute
446145,iphone xs max case jordan shoes us [SEP] ChyFS Cat iPhone Xs Max Phone Case Black Protective Case for iPhone Xs Max (2018 Release) (Basketball net) us ChyFS,substitute
531192,"black friday drinking shirts for women funny us [SEP] CafePress Professional Coach Drinking T Shirt 100% Cotton T-Shirt Black us CafePress This standard fit short-sleeve t-shirt features a comfortable crew neck and quality construction, making it the perfect graphic tee gift for both men and women. The soft fabric looks as good as it feels, and this tee is durable for everyday wear. Our cute, funny, and unique designs are printed professionally, and make great novelty gifts for him or her, young or old. With CafePress, your satisfaction is always our promise. Buy with confidence, as we offer easy returns and exchanges and a 100% money back guarantee. CASUAL & COMFORTABLE: Our Men's Value T-Shirt delivers classic comfort in a style all his own. With a traditional fit that runs true to size, at a price everyone loves, this is about to become your new favorite shirt!\nAN EVERYDAY ESSENTIAL: Our short-sleeve crewneck T-shirts are where casual comfort meets self-expression. Wash it in comfort: ; our tees won't shrink! They're made of comfortable, heavyweight 100% preshrunk cotton. Gray colors are made of a heavyweight cotton/polyester blend. A double-needle stitched bottom and hems add extra durability, and they're tagless for added comfort.\nPROFESSIONALLY PRINTED: Choose from several colors. Expertly printed with sharp, vivid designs, our tees are designed to show off your personality, passions and humor. Imported for final manufacture locally.\nMADE TO LAST: Our graphic T-shirts will make a statement everywhere you go. Easily machine wash cold with like colors, inside/out, then tumble dry low.\nIDEAL FOR YOURSELF & A UNIQUE GIFT: Surprise your husband, brother, dad, grandpa, son, uncle or friend, or order one just for you! Our value T-shirts make a u

In [12]:
train_df.head(2)

,input,label
0,"tack cleaner for horses us [SEP] Pack of 12 Tack Sponges us Hydra Sponge Pack of 12 Tack Sponges VALUE PACK of 12 Creative Hobbies Best Quality Synthetic sponges for Painting, Face Painting, Crafts, Pottery, Clay, Household Use & more!\nNEW IMPROVED FORMULA Synthetic material is soft and absorbent, finishes clay smoothly, is gentle on faces for face painting, and very absorbent.\nShipped compressed to save space, sponges expand by rinsing in warm water or exposure to air after unpacking.\nVERSATILE - Great for face painting, leather tack cleaning and conditioning, applying paint in household or craft projects to washing dishes.",complement
1,"brother p-touch pt-d210 us [SEP] Khanka Hard Travel Case Replacement for Brother P-Touch PT-D210 Label Maker us khanka DISCLAIMER: Product only include Khanka brand casing. Does NOT include other products as pictured. We are NOT sponsored by other brands. Khanka Carrying Storage Organizer Case Bag Replacement for Brother P-Touch PT-D210 Label Maker fits 1/2-Inch Standard Laminated Tape\nExtra Separate Compartment Mesh Pocket for 1/2-Inch Standard Laminated Tape\nHand Strap For keeping your devices safe and handy,Lightweight. Packed inclued: One Case\nMaterial: EVA Color: Black External Size:7*6.6*3.5 inch Weight: 220g Black",complement


In [13]:
train_df.shape[0], test_df.shape[0]

(625388, 156350)

In [14]:
from datasets import ClassLabel, Value, DatasetDict, Dataset

In [15]:
dataset = DatasetDict({
        'train': Dataset.from_pandas(train_df),
        'test': Dataset.from_pandas(test_df),
    }   
)

In [16]:
dataset['train']

Dataset({
    features: ['input', 'label'],
    num_rows: 625388
})

In [17]:
dataset['train'].features

{'input': Value(dtype='string', id=None),
 'label': Value(dtype='string', id=None)}

In [18]:
labels = ClassLabel(names=['irrelevant', 'exact', 'substitute', 'complement'])

In [19]:
dataset['train'].features, dataset['test'].features

({'input': Value(dtype='string', id=None),
  'label': Value(dtype='string', id=None)},
 {'input': Value(dtype='string', id=None),
  'label': Value(dtype='string', id=None)})

In [20]:
from transformers import AutoTokenizer,DataCollatorWithPadding
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/665 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

In [21]:
def tokenize_label_map(batch):
    tokens = tokenizer(batch['input'], max_length=128, padding='max_length', truncation=True)
    tokens['label'] = labels.str2int(batch['label'])
    return tokens
encoded_dataset = dataset.map(tokenize_label_map, batched=True)

  0%|          | 0/626 [00:00<?, ?ba/s]

  0%|          | 0/157 [00:00<?, ?ba/s]

In [22]:
encoded_dataset['train'].column_names

['input', 'label', 'input_ids', 'token_type_ids', 'attention_mask']

In [23]:
tokenized_datasets = encoded_dataset#.remove_columns(["input"])
tokenized_datasets = tokenized_datasets.rename_column("label", "labels")
tokenized_datasets.set_format("torch")

In [24]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['input', 'labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 625388
    })
    test: Dataset({
        features: ['input', 'labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 156350
    })
})

In [25]:
tokenized_datasets['test'].features

{'input': Value(dtype='string', id=None),
 'labels': Value(dtype='int64', id=None),
 'input_ids': Sequence(feature=Value(dtype='int32', id=None), length=-1, id=None),
 'token_type_ids': Sequence(feature=Value(dtype='int8', id=None), length=-1, id=None),
 'attention_mask': Sequence(feature=Value(dtype='int8', id=None), length=-1, id=None)}

In [26]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [27]:
from transformers import AutoModelForSequenceClassification
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=4)

Downloading:   0%|          | 0.00/51.7M [00:00<?, ?B/s]

Some weights of the model checkpoint at google/electra-small-discriminator were not used when initializing ElectraForSequenceClassification: ['discriminator_predictions.dense.weight', 'discriminator_predictions.dense_prediction.bias', 'discriminator_predictions.dense.bias', 'discriminator_predictions.dense_prediction.weight']
- This IS expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at google/electra-small-discriminator and are newly initialized: ['classifier

In [28]:
print(f"number of trainable parameters {sum(p.numel() for p in model.parameters() if p.requires_grad)}")

number of trainable parameters 13549828


In [29]:
from sklearn.metrics import precision_score, recall_score, accuracy_score, f1_score

In [30]:
from sklearn.metrics import precision_score, recall_score, accuracy_score, f1_score
import numpy as np
def compute_metrics(eval_preds):
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)
    metrics_dict = {
        'precision': precision_score(labels, predictions, average='micro'),
        'recall': recall_score(labels, predictions, average='micro'),
        'f1': f1_score(labels, predictions, average='micro'),
        'accuracy': accuracy_score(labels, predictions),
    }
    return metrics_dict

In [31]:
from transformers import Trainer
from transformers import TrainingArguments
import torch

training_args = TrainingArguments(evaluation_strategy="epoch", report_to="none", output_dir = '/kaggle/working/', load_best_model_at_end=True, save_total_limit=2, save_strategy='epoch', logging_strategy='steps', logging_steps=50, log_level='info', log_level_replica='info', fp16=torch.cuda.is_available())

trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

Using amp half precision backend


In [32]:
try:
    trainer.train()
except:
    print("Training stopped")

The following columns in the training set  don't have a corresponding argument in `ElectraForSequenceClassification.forward` and have been ignored: input.
/opt/conda/lib/python3.7/site-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use thePyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 625388
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 234522


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,0.955900,0.972925,0.582686,0.582686,0.582686,0.582686
2,0.898700,0.914978,0.619367,0.619367,0.619367,0.619367
3,0.825100,0.888461,0.636649,0.636649,0.636649,0.636649


/opt/conda/lib/python3.7/site-packages/transformers/trainer.py:1410: FutureWarning: Non-finite norm encountered in torch.nn.utils.clip_grad_norm_; continuing anyway. Note that the default behavior will change in a future release to error out if a non-finite total norm is encountered. At that point, setting error_if_nonfinite=false will be required to retain the old behavior.
  args.max_grad_norm,
The following columns in the evaluation set  don't have a corresponding argument in `ElectraForSequenceClassification.forward` and have been ignored: input.
***** Running Evaluation *****
  Num examples = 156350
  Batch size = 8
Saving model checkpoint to /kaggle/working/checkpoint-78174
Configuration saved in /kaggle/working/checkpoint-78174/config.json
Model weights saved in /kaggle/working/checkpoint-78174/pytorch_model.bin
tokenizer config file saved in /kaggle/working/checkpoint-78174/tokenizer_config.json
Special tokens file saved in /kaggle/working/checkpoint-78174/special_tokens_map.js

In [33]:
try:
    trainer.save_model('/kaggle/working/')
except:
    print("Trainer save stopped")

Saving model checkpoint to /kaggle/working/
Configuration saved in /kaggle/working/config.json
Model weights saved in /kaggle/working/pytorch_model.bin
tokenizer config file saved in /kaggle/working/tokenizer_config.json
Special tokens file saved in /kaggle/working/special_tokens_map.json
